In [12]:
!pip install opencv-python numpy matplotlib


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
!pip install imageio


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
import cv2
import imageio
import numpy as np
import time
from IPython.display import display, clear_output
from PIL import Image

# Function to display each frame inline
def show_frame_inline(frame):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(frame_rgb)
    clear_output(wait=True)
    display(img)

# Function to process a GIF and detect moving objects
def detect_moving_objects_from_gif(gif_path, save_output=False, output_path="detected_output.mp4"):
    # Load the GIF
    gif_reader = imageio.get_reader(gif_path)
    backSub = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=16, detectShadows=True)

    writer = None
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    frame_index = 0

    for frame_rgb in gif_reader:
        frame = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)
        fgmask = backSub.apply(frame)

        # Threshold and clean up the mask
        _, thresh = cv2.threshold(fgmask, 244, 255, cv2.THRESH_BINARY)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
        thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)

        # Find contours (moving objects)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for cnt in contours:
            if cv2.contourArea(cnt) < 300:  # Ignore small movements
                continue
            x, y, w, h = cv2.boundingRect(cnt)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Initialize video writer if saving
        if save_output and writer is None:
            h, w = frame.shape[:2]
            writer = cv2.VideoWriter(output_path, fourcc, 10, (w, h), True)

        # Show each frame inline
        show_frame_inline(frame)
        time.sleep(0.05)

        # Save to video if requested
        if save_output:
            writer.write(frame)

        frame_index += 1

    if save_output and writer:
        writer.release()
    clear_output(wait=True)
    print("Processing complete ✅")

In [19]:
detect_moving_objects_from_gif("Detect.gif", save_output=True, output_path="Detected_Result.mp4")

Processing complete ✅
